In [2]:
import pandas as pd
import datetime as dt
from pytz import timezone
from pytz import UTC

In [3]:
file_dir = './data/'
file_in = 'public_safety_center_MASTER.csv'
file_out = 'public_safety_center_MASTER_wrangled.csv'

In [4]:
header = ['AccountNo', 'MeterNo','Date', '10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59']

df = pd.read_csv(file_dir + file_in, header=None, skip_blank_lines=True)
df.columns = header

In [5]:
start_date = df['Date'][0]
start_time = '00:30'

end_date = df['Date'][len(df['Date'])-1]
end_time = '23:30'

#date_range = pd.date_range(start=start_date+' '+start_time, end='7/7/2015' + ' ' + '00:30', freq='30min', tz='US/Eastern')

date_range = pd.date_range(start=start_date+' '+start_time, end=end_date + ' ' + end_time, freq='30min', tz='US/Eastern')

print date_range

DatetimeIndex(['2009-01-08 00:30:00-05:00', '2009-01-08 01:00:00-05:00',
               '2009-01-08 01:30:00-05:00', '2009-01-08 02:00:00-05:00',
               '2009-01-08 02:30:00-05:00', '2009-01-08 03:00:00-05:00',
               '2009-01-08 03:30:00-05:00', '2009-01-08 04:00:00-05:00',
               '2009-01-08 04:30:00-05:00', '2009-01-08 05:00:00-05:00', 
               ...
               '2015-07-06 19:00:00-04:00', '2015-07-06 19:30:00-04:00',
               '2015-07-06 20:00:00-04:00', '2015-07-06 20:30:00-04:00',
               '2015-07-06 21:00:00-04:00', '2015-07-06 21:30:00-04:00',
               '2015-07-06 22:00:00-04:00', '2015-07-06 22:30:00-04:00',
               '2015-07-06 23:00:00-04:00', '2015-07-06 23:30:00-04:00'],
              dtype='datetime64[ns]', length=113805, freq='30T', tz='US/Eastern')


In [6]:
# Handling Daylight Savings Time

dst_start = ['3/8/2009', '3/14/2010', '3/13/2011', '3/11/2012', '3/10/2013', '3/9/2014', '3/8/2015']
df_dst_start = df[df['Date'].isin(dst_start)]
df_dst_start = df_dst_start.drop(['14', '15', '58', '59'], axis=1)

dst_end = ['11/1/2009', '11/7/2010', '11/6/2011', '11/4/2012', '11/3/2013', '11/2/2014', '11/1/2015']
df_dst_end = df[df['Date'].isin(dst_end)]
df_dst_end.columns = ['AccountNo', 'MeterNo','Date', '10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59']

In [7]:
df_dst_start_melt = pd.melt(df_dst_start, id_vars=['Date'], value_vars=['10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57'], var_name='Time', value_name='Use_kWh')
df_dst_end_melt = pd.melt(df_dst_end, id_vars=['Date'], value_vars=['10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59'], var_name='Time', value_name='Use_kWh')

In [8]:
df_dst_start_melt['Date'] = pd.DatetimeIndex(df_dst_start_melt['Date']).to_pydatetime()

for index, row in df_dst_start_melt[df_dst_start_melt['Time'] == '57'].iterrows():
    df_dst_start_melt['Date'].loc[index] = row['Date'] + pd.to_timedelta('1', unit='D')
    df_dst_start_melt['Time'].loc[index] = '0'
    
df_dst_end_melt['Date'] = pd.DatetimeIndex(df_dst_end_melt['Date']).to_pydatetime()

for index, row in df_dst_end_melt[df_dst_end_melt['Time'] == '59'].iterrows():
    df_dst_end_melt['Date'].loc[index] = row['Date'] + pd.to_timedelta('1', unit='D')
    df_dst_end_melt['Time'].loc[index] = '0'

/home/egeriicw/.venvs/nbserver/local/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
# Remove dst rows from main dataset
df = df[~df['Date'].isin(dst_start)]
df = df[~df['Date'].isin(dst_end)]

# Drop the dst columns from the end
df = df.drop(['58', '59'], axis=1)

In [10]:
# Melt main dataset
df_melt = pd.melt(df, id_vars=['Date'], value_vars=['10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57'], var_name='Time', value_name='Use_kWh')

In [11]:
df_melt['Date'] = pd.DatetimeIndex(df_melt['Date']).to_pydatetime()

for index, row in df_melt[df_melt['Time'] == '57'].iterrows():
    df_melt['Date'].loc[index] = row['Date'] + pd.to_timedelta('1', unit='D')
    df_melt['Time'].loc[index] = '0'

In [12]:
df_dst_melt = df_dst_start_melt.append(df_dst_end_melt)

In [13]:
df_melt_appended = df_melt.append(df_dst_melt)

In [14]:
df_melt_appended.sort(columns=['Date', 'Time'], inplace=True)

In [15]:
df_melt_appended = df_melt_appended[:-1].set_index(date_range)

In [16]:
df_melt_appended.reset_index(inplace=True)
df_melt_appended.columns.values[0] = 'DateTime'

In [17]:
df_melt_appended.to_csv(file_dir + 'df_melt_appended.csv', header=True)

In [18]:
df_melt_appended['Use_kWh'] = df_melt_appended['Use_kWh'].apply(lambda x: float(x))
df_melt_appended['P_kW'] = df_melt_appended['Use_kWh'].apply(lambda x: 2*float(x))

In [19]:
df_melt_appended['Date'] = df_melt_appended['DateTime'].apply(lambda x: x.date())
df_melt_appended['Time'] = df_melt_appended['DateTime'].apply(lambda x: x.time())
df_melt_appended['Month'] = df_melt_appended['DateTime'].apply(lambda x: x.month)
df_melt_appended['Day'] = df_melt_appended['DateTime'].apply(lambda x: x.day)
df_melt_appended['Year'] = df_melt_appended['DateTime'].apply(lambda x: x.year)
df_melt_appended['Hour'] = df_melt_appended['DateTime'].apply(lambda x: x.hour)
df_melt_appended['WeekDay'] = df_melt_appended['DateTime'].apply(lambda x: x.weekday())

In [20]:
# Insert fiscal year

def fiscal_year(val):
    if (val.month >= 7) and (val.month <=12):
        return val.year + 1
    else:
        return val.year
    
df_melt_appended['FiscalYear'] = df_melt_appended['DateTime'].apply(lambda x: fiscal_year(x))

In [21]:
df_melt_appended = df_melt_appended.ix[:, ['Date','Month', 'Day', 'Year', 'FiscalYear', 'WeekDay', 'Time', 'Hour', 'DateTime', 'Use_kWh', 'P_kW']]

In [22]:
df_melt_appended.to_csv(file_dir + file_out, header=True, columns=['Date','Month', 'Day', 'Year','FiscalYear','WeekDay', 'Time','Hour','DateTime','Use_kWh', 'P_kW'], index=True, mode='w')